<a href="https://colab.research.google.com/github/Satyake/Open-AI-LLM-Langchain-Projects/blob/main/Neural_Machine_Translation_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [1]:
!pip install transformers[sentencepiece]
!pip install sacrebleu rouge_score
!pip install py7zr
!pip install --upgrade accelerate
!pip install transformers accelerate
!pip install nltk
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=5e3fbd02befab418125bf7c53a0a300cd01fa9f5b84c324ebc4952a75cba2a91
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
!pip list

In [2]:
import os
import sys
import transformers
import tensorflow as tf
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import AdamWeightDecay

Google T5
Facebook M2M100
Google Cloud Translation API
Tune AI
Amazon Comprehend
Helisinki-NLP

In [3]:
path='Helsinki-NLP/opus-mt-en-hi'
model=TFAutoModelForSeq2SeqLM.from_pretrained(path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [4]:
tokenizer=AutoTokenizer.from_pretrained(path)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
dataset=load_dataset('cfilt/iitb-english-hindi')

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [7]:
dataset['train'][2]

{'translation': {'en': 'The default plugin layout for the bottom panel',
  'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'}}

In [8]:
dataset['test'][2]

{'translation': {'en': "The devices, which track every mile a motorist drives and transmit that information to bureaucrats, are at the center of a controversial attempt in Washington and state planning offices to overhaul the outdated system for funding America's major roads.",
  'hi': 'यह डिवाइस, जो मोटर-चालक द्वारा वाहन चलाए गए प्रत्येक मील को ट्रैक करती है तथा उस सूचना को अधिकारियों को संचारित करती है, आजकल अमेरिका की प्रमुख सड़कों का वित्त-पोषण करने के लिए पुराने हो चुके सिस्टम का जीर्णोद्धार करने के लिए वाशिंगटन और राज्य नियोजन कार्यालय के लिए एक विवादास्पद प्रयास का मुद्दा बन चुका है।'}}

In [9]:
tokenizer('There is Sunny, who is teaching data sci')

{'input_ids': [547, 23, 11647, 11453, 2, 66, 23, 1762, 1195, 13948, 624, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
tokenizer(['hello this is sunny','this is another sunny'])

{'input_ids': [[39915, 90, 23, 41670, 0], [90, 23, 414, 41670, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [11]:
with tokenizer.as_target_tokenizer():
  print(tokenizer('Hi'))

{'input_ids': [2204, 624, 0], 'attention_mask': [1, 1, 1]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
dataset['train']['translation']

In [16]:
for ex in dataset['train']['translation'][0:1]:
  print(ex['en'])


Give your application an accessibility workout


In [12]:
max_input_length=128
source_lang='en'
target_lang='hi'
def preprocess_function(sentence):
  inputs=[ex[source_lang] for ex in sentence['translation']]
  targets=[ex[target_lang] for ex in sentence['translation']]
  model_inputs=tokenizer(inputs, max_length=max_input_length, truncation=True)

  with tokenizer.as_target_tokenizer():
    labels=tokenizer(targets, max_length=128, truncation=True)

  model_inputs['labels']=labels['input_ids']
  return model_inputs

In [13]:
tokenized_data=dataset.map(preprocess_function, batched=True)

In [14]:
tokenized_data['train']['input_ids'][0]

[3872, 85, 2501, 132, 15441, 36398, 0]

In [15]:
batch_size=32
learning_rate=2e-5
weight_decay=0.01
num_train_epochs=1
data_collator=DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='tf')

In [16]:
train_dataset=model.prepare_tf_dataset(
    tokenized_data['test'],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator
)
validation_dataset=model.prepare_tf_dataset(
    tokenized_data['validation'],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator
)

In [19]:
optimizer=AdamWeightDecay(lr=learning_rate, decay=weight_decay)

In [20]:
model.compile(optimizer=optimizer)

In [21]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=num_train_epochs)

78/78 [==============================] - 2138s 27s/step - loss: 3.8291 - val_loss: 4.0608


In [22]:
model.save_pretrained('tf_model/')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


In [23]:
input_text="i am learning coding. How are you "

In [24]:

tokenizer=AutoTokenizer.from_pretrained(path)
#model=TFAutoModelForSeq2SeqLM.from_pretrained('tf_model/')

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [25]:
tokenized=tokenizer([input_text], return_tensors='np')

In [26]:
out=model.generate(**tokenized, max_length=128)

In [27]:
print(out)

tf.Tensor(
[[61949   104  1780   209   492   254     2   280   118    57   160    28
      0]], shape=(1, 13), dtype=int32)


In [28]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0], skip_special_tokens=True))

मैं पढ़ने जा रही हूँ, कैसे आप कर रहे हैं


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
